In [ ]:
import numpy as np
from scipy.optimize import brentq
import pandas as pd
np.set_printoptions(precision=18)

def market_discount_factors(spot_rates):
    """
    Calculate market discount factors from spot rates.
    Parameters:
    spot_rates (list or array-like): A list of spot rates (in decimals) for different maturities.

    Returns:
    numpy.ndarray: An array of discount factors corresponding to the spot rates.

    Description:
    This function computes the discount factors for a series of spot rates. 
    The discount factor represents the present value of one unit of currency to be received at a future date. 
    It is calculated as the reciprocal of (1 + spot rate) raised to the power of the maturity period.
    """
    spot = np.asarray(spot_rates, dtype=np.float64)
    n = len(spot)
    discounts = np.empty(n, dtype=np.float64)
    for k in range(n):
        discounts[k] = 1.0 / np.power(1.0 + spot[k], k + 1)
    return discounts

def price_zcb_on_tree(a_list, b, t):
    """
    Calculate the price of a zero-coupon bond (ZCB) using a binomial tree model.

    Parameters:
    a_list (list or array-like): A list or array of parameters a_1, a_2, ..., a_t (length >= t) 
                                 that define the short-rate lattice.
    b (float): A scalar parameter that determines the exponential growth of short rates in the lattice.
    t (int): The maturity of the zero-coupon bond (1 <= t <= len(a_list)).

    Returns:
    float: The model price of the zero-coupon bond at time 0 for a unit face value.

    Description:
    This function computes the price of a zero-coupon bond using a binomial tree model for short rates. 
    The short rates are determined by the parameters `a_list` and `b`, and the bond price is calculated 
    by backward induction starting from the bond's payoff at maturity. At each step, the bond price is 
    computed as the discounted expected value of the bond prices at the next time step, assuming equal 
    probabilities for upward and downward movements in the short-rate lattice.
    """
    # Initialize terminal prices at maturity
    P_next = np.ones(t+1)  # Payoff = 1 at maturity

    # Backward induction to calculate bond price at time 0
    for k in range(t, 0, -1):
        a_k = a_list[k-1]
        # Calculate short rates r(k,j) for j=0..k
        r_k = a_k * np.exp(b * np.arange(0, k+1))
        P_prev = np.empty(k)
        # Compute bond prices at the previous time step
        for j in range(k):
            P_prev[j] = 0.5 * (P_next[j] / (1.0 + r_k[j])) + 0.5 * (P_next[j+1] / (1.0 + r_k[j+1]))
        P_next = P_prev

    # Return the bond price at time 0
    return P_next[0]

def calibrate_bdt_a(spot_rates, b=0.05, tol=1e-8):
    """
    Calibrate the parameters 'a' for the Black-Derman-Toy (BDT) short-rate model.

    Parameters:
    - spot_rates (list or array-like): A list of spot rates (in decimals) for different maturities.
                                       The length of this list determines the number of time steps in the model.
    - b (float, optional): A scalar parameter that determines the exponential growth of short rates in the lattice.
                           Default value is 0.05.
    - tol (float, optional): The tolerance for the root-finding algorithm. It determines the acceptable error
                             in the calibration process. Default value is 1e-8.

    Returns:
    - numpy.ndarray: An array of calibrated 'a' parameters for the BDT model. The length of this array matches
                     the number of spot rates provided. These parameters are used to construct the short-rate lattice.

    Description:
    This function calibrates the 'a' parameters for the BDT short-rate model by ensuring that the model prices
    of zero-coupon bonds (ZCBs) match the market prices derived from the given spot rates. The calibration is
    performed iteratively for each time step, starting from the first maturity and moving to the last.

    For each time step 'k', the function:
    1. Defines an objective function `f(a)` that calculates the difference between the model price of a ZCB
       (using the current guess for 'a') and the market price of the ZCB.
    2. Uses the Brent root-finding method to find the value of 'a' that minimizes the objective function.
    3. Updates the array of 'a' parameters with the calibrated value for the current time step.

    The calibrated 'a' parameters are then returned as a numpy array.
    """
    # spot_rates: list of length n (decimals)
    n = len(spot_rates)
    P_market = market_discount_factors(spot_rates)  # length n
    a_cal = np.zeros(n)
    for k in range(1, n+1):
        # objective: f(a) = model_price(0,k) - P_market[k-1]
        def f(a_val):
            a_try = a_cal.copy()
            a_try[k-1] = a_val
            return price_zcb_on_tree(a_try, b, k) - P_market[k-1]

        # bracket search
        low, high = 1e-12, 0.5
        f_low, f_high = f(low), f(high)
        # expand high until sign change or cap
        ntries = 0
        while f_low * f_high > 0 and ntries < 80:
            high *= 2.0
            f_high = f(high)
            ntries += 1
        if f_low * f_high > 0:
            raise RuntimeError(f"Could not bracket root for a_{k}: f(low)={f_low}, f(high)={f_high}")

        a_root = brentq(f, low, high, xtol=1e-12, rtol=1e-12, maxiter=200)
        # ensure residual small enough
        if abs(f(a_root)) > tol:
            raise RuntimeError(f"Calibration failed at k={k}, residual {abs(f(a_root))} > {tol}")
        a_cal[k-1] = a_root
    return a_cal

"""
Brent’s method (brentq in SciPy) is a bracketed root‑finder that mixes bisection, the secant method and inverse‑quadratic interpolation: it keeps a sign‑changing interval for guaranteed convergence and uses faster interpolation steps when safe, giving both reliability and near‑secant speed.

Pros

Robust: guarantees convergence for continuous functions if you can bracket a sign change.
Fast in practice: switches to interpolatory steps when appropriate (typically faster than pure bisection).
Derivative‑free: works when derivatives are unavailable or noisy.
Cons

Requires a bracket where f(low)*f(high) < 0 (you must find a sign change).
Can be slower than Newton when a good derivative and initial guess are available.
Fails on discontinuous functions or when multiple roots lie inside the bracket (you get one root only).

Alternatives

Newton–Raphson (fast quadratic convergence, needs derivative and good initial guess).
Secant method (derivative‑free, faster than bisection but not guaranteed).
Bisection (very robust, but slow).
Ridder’s method or regula falsi (other bracketed schemes with different tradeoffs).
Global/root‑finding libraries or scalar optimizers if bracketing is hard (e.g., brenth, scipy.optimize.root with methods that use Jacobians).
"""

In [120]:
def build_bdt_short_rate_lattice(a_params, b):
    """
    Build a right-aligned BDT short-rate lattice as a pandas DataFrame.
    a_params: 1D array-like of a_1..a_n (length n)
    b: scalar
    returns: n x n DataFrame with NaN in unused entries
    """
    n = len(a_params)
    rates = np.full((n, n), np.nan, dtype=float)
    for col in range(n):
        for j in range(col + 1):
            row = n - 1 - j
            rates[row, col] = a_params[col] * np.exp(b * j)
    return pd.DataFrame(rates, index=range(n), columns=range(n))

In [121]:
def compute_elementary_prices(lattice, q):
    """
    Compute the elementary prices (state prices) for a given short-rate lattice.

    Parameters:
    - lattice (pd.DataFrame): A pandas DataFrame representing the short-rate lattice. 
                              Each cell contains the short rate (in percentage) at a specific time and state.
    - q (float): The risk-neutral probability of an upward movement in the short-rate lattice (0 <= q <= 1).

    Returns:
    - pd.DataFrame: A pandas DataFrame containing the computed elementary prices. 
                    Each cell represents the price of a unit payoff at a specific time and state.
                    The last row ('Model Prices') contains the sum of elementary prices for each time step, 
                    which corresponds to the model's discount factors.

    Purpose:
    This function calculates the elementary prices (state prices) for a given short-rate lattice using a backward induction approach. 
    Elementary prices represent the present value of a unit payoff in a specific state at a specific time. 
    These prices are used in financial modeling to value derivatives, bonds, and other financial instruments under the risk-neutral measure.
    """
    rows, cols, size = lattice.shape[0] + 1, lattice.shape[1] + 1, lattice.shape[0] + 1
    A = pd.DataFrame(np.nan, index=range(rows), columns=range(cols), dtype=float)
    A.iat[rows - 1, 0] = 1.0  # time 0 state

    for i in range(1, size):  # columns (time steps)
        for k in range(size - 1, size - i - 2, -1):  # rows (states)
            if k == size - 1:
                # Compute elementary price for the highest state
                A.iat[k, i] = (1 - q) * A.iat[k, i - 1] / (1 + lattice.iat[size - 2, i - 1] / 100)
            elif (k != size - 1 and k + i == size - 1):
                # Compute elementary price for the lowest state
                A.iat[k, i] = (q) * A.iat[k + 1, i - 1] / (1 + lattice.iat[size - 2, i - 1] / 100)
            elif (k != size - 1 and k + i != size - 1):
                # Compute elementary price for intermediate states
                A.iat[k, i] = (q) * A.iat[k + 1, i - 1] / (1 + lattice.iat[size - 2, i - 1] / 100) + \
                              (1 - q) * A.iat[k, i - 1] / (1 + lattice.iat[size - 2, i - 1] / 100)

    # Compute the sum of elementary prices for each time step
    col_sums = A.sum(axis=0)
    A.loc['Model Prices'] = col_sums
    return A

In [148]:
def price_payer_swaption(short_rate_lattice, swaption_fixed_rate, option_expiration, swap_maturity, principal, q):
    """
    Calculate the price of a payer swaption using a binomial tree model.

    Parameters:
    - short_rate_lattice (pd.DataFrame): A pandas DataFrame representing the short-rate lattice. 
                                         Each cell contains the short rate (in percentage) at a specific time and state.
    - swaption_fixed_rate (float): The fixed rate of the swaption contract.
    - option_expiration (int): The expiration time of the swaption (in time steps).
    - swap_maturity (int): The maturity of the underlying swap (in time steps).
    - principal (float): The notional principal amount of the swaption.
    - q (float): The risk-neutral probability of an upward movement in the short-rate lattice (0 <= q <= 1).

    Returns:
    - float: The value of the payer swaption at time 0.

    Purpose:
    This function calculates the value of a payer swaption using a binomial tree model for short rates. 
    A payer swaption gives the holder the right (but not the obligation) to enter into a swap where they pay 
    the fixed rate (swaption_fixed_rate) and receive the floating rate. The function uses backward induction 
    to compute the value of the swaption, starting from the payoff at maturity and working back to time 0.

    The calculation involves:
    1. Constructing a swap lattice to compute the value of the underlying swap at each node of the tree.
    2. Applying the option payoff condition at the expiration of the swaption.
    3. Using backward induction to calculate the value of the swaption at earlier time steps, considering 
       the risk-neutral probabilities and discounting based on the short rates.

    The final value at time 0 is multiplied by the principal to obtain the monetary value of the swaption.
    """
    swap_lattice = pd.DataFrame(np.nan, index=range(swap_maturity), columns=range(swap_maturity), dtype=float)
    swap_lattice.iloc[:, -1] = (((short_rate_lattice[swap_maturity-1].dropna())/100 - swaption_fixed_rate) / (1 + (short_rate_lattice[swap_maturity-1].dropna())/100)).reset_index(drop=True)
    short_rate_lattice_trim = short_rate_lattice.iloc[-swap_maturity:, :swap_maturity].reset_index(drop=True)
    print("Short rate lattice trimmed")
    print(short_rate_lattice_trim)
    
    for i in range(swap_maturity - 2, -1, -1):  # Backward induction for swap lattice
        for k in range(swap_maturity - 1, swap_maturity - 2 - i, -1):
            swap_lattice.iat[k, i] = (short_rate_lattice_trim.iat[k, i]/100 - swaption_fixed_rate) / (1 + short_rate_lattice_trim.iat[k, i]/100) + \
                                     (q * swap_lattice.iat[k-1, i+1] + (1-q) * swap_lattice.iat[k, i+1]) / (1 + short_rate_lattice_trim.iat[k, i]/100)
    print("Swap lattice")
    print(swap_lattice)
    swap_price = swap_lattice.iloc[-(option_expiration+1):, :(option_expiration+1)].reset_index(drop=True)
    swap_price.iloc[:, -1] = swap_price.iloc[:, -1].apply(lambda x: max(x, 0))  # Apply option payoff condition
    print("Swap price after applying payoff condition")
    print(swap_price)
    short_rate_lattice_trim_swap = short_rate_lattice.iloc[-(option_expiration+1):, :option_expiration+1].reset_index(drop=True)

    for i in range(option_expiration - 1, -1, -1):  # Backward induction for swaption pricing
        for k in range(option_expiration, option_expiration - i - 1, -1):
            swap_price.iat[k, i] = (q * swap_price.iat[k-1, i+1] + (1-q) * swap_price.iat[k, i+1]) / (1 + short_rate_lattice_trim_swap.iat[k, i]/100)
    print("Final swap price lattice after backward induction")
    print(swap_price)
    payer_swaption_value = swap_price.iat[-1, 0] * principal
    print(f"The value of the swaption for payer is {payer_swaption_value}")
    return payer_swaption_value

In [149]:
# test 1
spot_rates = [
    7.299995921, 7.921105449, 9.021170353, 9.435721827, 12.13023009, 
    11.71925372, 12.85017382, 12.56598542, 12.9185185, 15.19509232, 
    14.53647554, 15.63614692, 15.15400561, 13.44790071
]
short_rate_lattice = build_bdt_short_rate_lattice(spot_rates, b=0.005)
print("Short rate lattice")
print(short_rate_lattice)
elementary_price_lattice = compute_elementary_prices(short_rate_lattice, q=0.5)
print("Elementary price lattice")
print(elementary_price_lattice)
calibrated_params = calibrate_bdt_a(spot_rates, b=0.005)
print("Calibrated parameters")
print(calibrated_params)

#inputs for swaption pricing
swaption_fixed_rate = 0.1165
option_expiration = 2
swap_maturity = 10
option_strike = 0
principal = 1000000
q = 0.5

price_payer_swaption(short_rate_lattice, swaption_fixed_rate, option_expiration, swap_maturity, principal, q)

Short rate lattice
          0         1         2         3          4          5          6   \
0        NaN       NaN       NaN       NaN        NaN        NaN        NaN   
1        NaN       NaN       NaN       NaN        NaN        NaN        NaN   
2        NaN       NaN       NaN       NaN        NaN        NaN        NaN   
3        NaN       NaN       NaN       NaN        NaN        NaN        NaN   
4        NaN       NaN       NaN       NaN        NaN        NaN        NaN   
5        NaN       NaN       NaN       NaN        NaN        NaN        NaN   
6        NaN       NaN       NaN       NaN        NaN        NaN        NaN   
7        NaN       NaN       NaN       NaN        NaN        NaN  13.241520   
8        NaN       NaN       NaN       NaN        NaN  12.015928  13.175478   
9        NaN       NaN       NaN       NaN  12.375277  11.955998  13.109765   
10       NaN       NaN       NaN  9.578325  12.313555  11.896368  13.044379   
11       NaN       NaN  9.111835 

1339.0928272797648

In [ ]:
# test 2
spot_rates = [
    2.999998512448560, 3.040459014315700, 3.069777369043150, 3.088995878545480, 
    3.099161927924590, 3.101111369601270, 2.836631367683650, 2.766862296160510, 
    2.697461348921240, 2.628443022603120
]
short_rate_lattice = build_bdt_short_rate_lattice(spot_rates, b=0.1)
print("Short rate lattice")
print(short_rate_lattice)
elementary_price_lattice = compute_elementary_prices(short_rate_lattice, q=0.5)
print("Elementary prices")
print(elementary_price_lattice)
calibrated_params = calibrate_bdt_a(spot_rates, b=0.005)
print("Calibrated parameters")
print(calibrated_params)

#inputs for swaption pricing
swaption_fixed_rate = 0.039
option_expiration = 3
swap_maturity = 10
option_strike = 0
principal = 1000000
q = 0.5

price_payer_swaption(short_rate_lattice, swaption_fixed_rate, option_expiration, swap_maturity, principal, q)

Short rate lattice trimmed
          0         1         2         3         4         5         6  \
0       NaN       NaN       NaN       NaN       NaN       NaN       NaN   
1       NaN       NaN       NaN       NaN       NaN       NaN       NaN   
2       NaN       NaN       NaN       NaN       NaN       NaN       NaN   
3       NaN       NaN       NaN       NaN       NaN       NaN  5.168679   
4       NaN       NaN       NaN       NaN       NaN  5.112868  4.676814   
5       NaN       NaN       NaN       NaN  4.623406  4.626315  4.231757   
6       NaN       NaN       NaN  4.169708  4.183431  4.186062  3.829052   
7       NaN       NaN  3.749435  3.772908  3.785325  3.787706  3.464669   
8       NaN  3.360227  3.392629  3.413868  3.425104  3.427258  3.134962   
9  2.999999  3.040459  3.069777  3.088996  3.099162  3.101111  2.836631   

          7         8         9  
0       NaN       NaN  6.464927  
1       NaN  6.003311  5.849708  
2  5.571776  5.432020  5.293034  
3  5.041552

8096.727362457942

In [143]:
# test 3
spot_rates = [
    2.999998019381450, 3.120171778139800, 3.232631109364390, 3.337708570299540, 
    3.435703887194930, 3.526946385802580, 3.309542134678470, 3.311310295349930, 
    3.311053139801530, 3.308924096566770
]
short_rate_lattice = build_bdt_short_rate_lattice(spot_rates, b=0.05)
#print("Short rate lattice")
#print(short_rate_lattice)
elementary_price_lattice = compute_elementary_prices(short_rate_lattice, q=0.5)
#print("Elementary prices")
#print(elementary_price_lattice)
calibrated_params = calibrate_bdt_a(spot_rates, b=0.005)
#print("Calibrated parameters")
#print(calibrated_params)

#inputs for swaption pricing
swaption_fixed_rate = 0.039
option_expiration = 3
swap_maturity = 10
option_strike = 0
principal = 1000000
q = 0.5

price_payer_swaption(short_rate_lattice, swaption_fixed_rate, option_expiration, swap_maturity, principal, q)

Short rate lattice trimmed
          0         1         2         3         4         5         6  \
0       NaN       NaN       NaN       NaN       NaN       NaN       NaN   
1       NaN       NaN       NaN       NaN       NaN       NaN       NaN   
2       NaN       NaN       NaN       NaN       NaN       NaN       NaN   
3       NaN       NaN       NaN       NaN       NaN       NaN  4.467415   
4       NaN       NaN       NaN       NaN       NaN  4.528689  4.249536   
5       NaN       NaN       NaN       NaN  4.196378  4.307822  4.042284   
6       NaN       NaN       NaN  3.877864  3.991718  4.097727  3.845139   
7       NaN       NaN  3.572610  3.688738  3.797040  3.897879  3.657610   
8       NaN  3.280146  3.398372  3.508837  3.611856  3.707777  3.479226   
9  2.999998  3.120172  3.232631  3.337709  3.435704  3.526946  3.309542   

          7         8         9  
0       NaN       NaN  5.189426  
1       NaN  4.939511  4.936335  
2  4.698973  4.698608  4.695587  
3  4.469801

4102.145445457145